In [1]:
'''
# 라이브러리 정의
 - 시스템 라이브러리
 - 오디오 관련 라이브러리
 - 연산 및 Plot 관련 라이브러리 
 - 샤용자 라이브러리
'''

# Import Systems 
import struct
import io
import os
import sys
import math
import platform

# Import Audio
import pyaudio
import sounddevice
import librosa
import soundfile

import numpy as np
import scipy
import scipy.signal as sig
import matplotlib.pyplot as plt

# User Libraries
import pyOssWavfile
import pyRoomAcoustic as room
import pyOssDebug as dbg
import pyOssFilter
import pyOssPyAudio


In [2]:
set_device_type = None

if 'darwin' in platform.system():
    IDX = pyaudio.paCoreAudio           # = 5
elif 'linux' in platform.system():
    IDX = pyaudio.paALSA
else:
    if set_device_type == 'asio':
        IDX = pyaudio.paASIO            # = 3
    elif set_device_type == 'wasapi':
        IDX = pyaudio.paWASAPI          # = 13
    elif set_device_type == 'wdm':
        IDX = pyaudio.paWDMKS           # = 11
    else:
        IDX = pyaudio.paMME             # = 2

print('Index of Device Type:', IDX)

# List of Device Names IDX 
dev_name = pyOssPyAudio.ju_get_device_name(IDX)
print(dev_name)


Index of Device Type: 2
['Microsoft 사운드 매퍼 - Input', 'Headset Microphone(Cirrus Logic', 'Internal Microphone(Cirrus Logi', 'Microsoft 사운드 매퍼 - Output', '헤드셋(Cirrus Logic CS8409 (AB 16)', '스피커(Cirrus Logic CS8409 (AB 16)']


In [4]:
# 장치선택

tgt_dev_in = dev_name[0]
tgt_dev_out = dev_name[3]

dev_in = pyOssPyAudio.ju_get_device_info(IDX, tgt_dev_in)
dev_in_index = dev_in['index']
dev_in_name = dev_in['name']
dev_in_ch = dev_in['maxInputChannels']
dev_in_fs = int(dev_in['defaultSampleRate'])

dev_out = pyOssPyAudio.ju_get_device_info(IDX, tgt_dev_out)
dev_out_index = dev_out['index']
dev_out_name = dev_out['name']
dev_out_ch = dev_out['maxInputChannels']
dev_out_fs = int(dev_out['defaultSampleRate'])


# ASIO 장치 선택
# tgt_asio_dev_name = 'ASIO4ALL v2'
tgt_asio_dev_name = 'Yamaha Steinberg USB ASIO'
# tgt_asio_dev_name = 'Steinberg USB ASIO'

dev_asio = pyOssPyAudio.ju_get_device_info(IDX, tgt_asio_dev_name)

dev_asio_index = dev_asio['index']
dev_asio_name = dev_asio['name']
dev_asio_ch_in = dev_asio['maxInputChannels']
dev_asio_fs = dev_asio['defaultSampleRate']

print('dev_asio: ', dev_asio)
print(dev_asio_index, dev_asio_name, dev_asio_ch_in, dev_asio_fs)


In [ ]:
# Record & save audio data from Mic using ASIO Device

pa = pyaudio.PyAudio()

FORMAT = pa.get_format_from_width(2)
CH = dev_asio_ch_in
CHUNK = 1024 
RATE = np.int(dev_asio_fs)            # device 정보에서 읽어오면 됨
BUFFER = CHUNK * 4
RECORD_SECONDS = 10

print(pa.is_format_supported(RATE, 
            input_device=dev_asio_index,
            input_channels=CH,
            input_format=FORMAT,
     ))

stream1 = pa.open(
    format = FORMAT,
    channels = CH,
    rate = RATE,
    input = True,
    output = False,
    frames_per_buffer = BUFFER,
    input_device_index = dev_asio_index
)

# stream2 = pa.open(
#     format = FORMAT,
#     channels = 2,
#     rate = RATE,
#     input = True,
#     output = False,
#     frames_per_buffer = BUFFER,
#     input_device_index = dev_asio_index
# )


# Record input audio data
print("* recording")

frames1 = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data1 = stream1.read(CHUNK)
    frames1.append(data1)

print("* done recording")

In [ ]:
# Stream Stop Process 

stream1.stop_stream()
stream1.close()
pa.terminate()          #


# Not really sure what b'' means in BYTE STRING but numpy needs it 
# just like wave did...
framesAll = b''.join(frames1)

# Use numpy to format data and reshape.  
# PyAudio output from stream.read() is interlaced.
result = np.fromstring(framesAll, dtype=np.int16)
chunk_length = np.int(len(result) / CH)
result = np.reshape(result, (chunk_length, CH))


# Write multi-channel .wav file with SciPy
pyOssWavfile.write('pyaudio_rec.wav', RATE, result)
print("* done save wav file")